### Notebook for Capstone project

Every task for current assignment in this notebook, so you don't need to download another notebooks. Links on coursera all the same.

#### First part

In [93]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import geocoder
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import requests
import json
from pandas.io.json import json_normalize

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_data = pd.read_html(url)

pd.read_html reads all tables on page. 1st table (0 index) is a table of interest

In [3]:
raw_data = raw_data[0]
raw_data.shape

(180, 3)

In [4]:
(raw_data.Borough == 'Not assigned').sum() == (raw_data.Neighbourhood == 'Not assigned').sum()

True

If True, there are no rows with assigned Borough and not assigned Neighbourhood

Next, we'll use only cols with assigned Borough

In [5]:
data = raw_data[raw_data.Borough!='Not assigned']

To get rid of repeating postal codes we group table by this col and sum through groups

In [6]:
data = data.groupby('Postal Code').sum().reset_index()

In [7]:
data.shape

(103, 3)

In [8]:
data

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [10]:
'''#loop until you get the coordinates
postal_code = 'M1B'
lat_lng_coords = None
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]'''

"#loop until you get the coordinates\npostal_code = 'M1B'\nlat_lng_coords = None\nwhile(lat_lng_coords is None):\n    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n    lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]"

geocoder doesn't work

In [11]:
stamps = pd.read_csv('https://cocl.us/Geospatial_data')

In [12]:
stamps.sort_values('Postal Code', inplace=True)
data.sort_values('Postal Code', inplace=True)

Not shure wether sorting is nessesary 

In [13]:
data = data.set_index('Postal Code').join(stamps.set_index('Postal Code'))
data.reset_index(inplace=True)

In [15]:
latitude = 43.651070
longitude = -79.347015
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighbourhood']):
    neighborhood = neighborhood.split(',')
    neighborhood = neighborhood[0]
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Kindly ask you to use your own credentials

In [16]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: E0VRZ503WXG1AXTVACAFDH4WTRBN15FJ1H4AWE4JUQZJ4YJD
CLIENT_SECRET:HQJVJGAWXMUZKTEUIE4B3MK1GCBQDDI0HKDWPYHLSFZPYGG0


Check if everythig works fine on a single example

In [17]:
lat = data.loc[0, 'Latitude']
lon = data.loc[0, 'Longitude']
name = data.loc[0, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(name, 
                                                               lat, 
                                                               lon))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [18]:
LIMIT = 120
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    lon,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=E0VRZ503WXG1AXTVACAFDH4WTRBN15FJ1H4AWE4JUQZJ4YJD&client_secret=HQJVJGAWXMUZKTEUIE4B3MK1GCBQDDI0HKDWPYHLSFZPYGG0&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=120'

In [19]:
results = requests.get(url).json()
#results[]

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-21-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


Most part of analysis is from coursera lab, so I don't think that comments are necsesary

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
venues_to_explore = getNearbyVenues(data['Neighbourhood'], data['Latitude'], data['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [24]:
venues_to_explore.columns

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [25]:
venues_to_explore.shape

(2138, 7)

In [26]:
venues_to_explore.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
"Willowdale, Willowdale East",35,35,35,35,35,35
"Willowdale, Willowdale West",4,4,4,4,4,4
Woburn,4,4,4,4,4,4


In [27]:
print('There are {} uniques categories.'.format(len(venues_to_explore['Venue Category'].unique())))

There are 274 uniques categories.


In [35]:
venues_one_hot = pd.get_dummies(venues_to_explore[['Venue Category']], prefix="", prefix_sep="")
venues_one_hot['Neighborhood_lst'] = venues_to_explore['Neighborhood']

In [42]:
fixed_cols = [venues_one_hot.columns[-1]] + list(venues_one_hot.columns[:-1])
venues_one_hot = venues_one_hot[fixed_cols]

In [60]:
toronto_grouped = venues_one_hot.groupby('Neighborhood_lst').mean().reset_index()

I've grouped all airport features

In [61]:
toronto_grouped.loc[:,['Airport',
       'Airport Food Court', 'Airport Gate', 'Airport Lounge',
       'Airport Service', 'Airport Terminal']].sum(axis=1)

toronto_grouped['Airport'] = toronto_grouped.loc[:,['Airport',
       'Airport Food Court', 'Airport Gate', 'Airport Lounge',
       'Airport Service', 'Airport Terminal']].sum(axis=1)
cols_drop = ['Airport Food Court', 'Airport Gate','Airport Lounge', 'Airport Service','Airport Terminal']
toronto_grouped = toronto_grouped.drop(cols_drop, axis=1)


In [64]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood_lst']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood_lst'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1  Latin American Restaurant  0.25
2                     Lounge  0.25
3               Skating Rink  0.25
4                      Motel  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.33
1             Gym  0.17
2     Coffee Shop  0.17
3             Pub  0.17
4  Sandwich Place  0.17


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0               Bank  0.10
1        Coffee Shop  0.10
2        Bridal Shop  0.05
3        Pizza Place  0.05
4  Convenience Store  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4          Music Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2      Sandwich Place  0.09
3     Thai Restaura

                        venue  freq
0              Baseball Field   0.5
1                Home Service   0.5
2  Modern European Restaurant   0.0
3                      Museum   0.0
4               Movie Theater   0.0


----Parkdale, Roncesvalles----
                         venue  freq
0               Breakfast Spot  0.14
1                    Gift Shop  0.14
2  Eastern European Restaurant  0.07
3             Cuban Restaurant  0.07
4                Movie Theater  0.07


----Parkview Hill, Woodbine Gardens----
                  venue  freq
0           Pizza Place  0.18
1        Breakfast Spot  0.09
2  Gym / Fitness Center  0.09
3                  Bank  0.09
4                  Café  0.09


----Parkwoods----
                  venue  freq
0     Food & Drink Shop  0.25
1                  Pool  0.25
2                  Park  0.25
3  Fast Food Restaurant  0.25
4     Accessories Store  0.00


----Queen's Park, Ontario Provincial Government----
                venue  freq
0         Coffee Shop  0.

In [73]:
def return_most_common_venues(row, num_top_venues):
    #row_categories = row.iloc[1:]
    row_categories_sorted = row.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood_lst']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, 1:], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Lounge,Latin American Restaurant,Skating Rink,Escape Room,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store
1,"Alderwood, Long Branch",Pizza Place,Pub,Sandwich Place,Gym,Coffee Shop,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Deli / Bodega,Shopping Mall,Sandwich Place,Middle Eastern Restaurant,Supermarket,Sushi Restaurant,Restaurant
3,Bayview Village,Bank,Japanese Restaurant,Chinese Restaurant,Café,Yoga Studio,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store
4,"Bedford Park, Lawrence Manor East",Thai Restaurant,Coffee Shop,Sandwich Place,Italian Restaurant,Greek Restaurant,Comfort Food Restaurant,Grocery Store,Pharmacy,Pizza Place,Pub


In [78]:
num_clusters = 6
toronto_clustering = toronto_grouped.drop('Neighborhood_lst', axis=1)

In [89]:
KM = KMeans(n_clusters=num_clusters, random_state=1)
KM.fit(toronto_clustering)
KM.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [91]:
#neighborhoods_venues_sorted.insert(0, 'Cluster labels', KM.labels_)
data_merged = data
data_merged = data_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
data_merged.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Print Shop,Fast Food Restaurant,Escape Room,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Yoga Studio
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Bar,Yoga Studio,Escape Room,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Fish Market
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Bank,Mexican Restaurant,Breakfast Spot,Restaurant,Rental Car Location,Electronics Store,Medical Center,Intersection,Event Space,Ethiopian Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Pharmacy,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Discount Store,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Lounge,Athletics & Sports,Fried Chicken Joint,Caribbean Restaurant,Gas Station,Bakery,Thai Restaurant,Bank,Hakka Restaurant,Cupcake Shop


In [102]:
data_merged.fillna(0, inplace=True)

In [103]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_merged['Latitude'], data_merged['Longitude'], data_merged['Neighbourhood'], data_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

This is an axample of not so good clustering. There is one general class and five minor classes (aka outliers). Probabply happend due to bad feature analysis. I've tried to merge some characteristics (only about airports, honestly). But I think thorough feature analisis is 'out of scope' of this lab.